In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import sleep
from random import randint
import time
from selenium import webdriver
from urllib.parse import urljoin

In [8]:
##### Web scrapper for infinite scrolling page #####
def scraper(url, scroll_time):
    driver = webdriver.Chrome(executable_path=r"E:\Chromedriver\chromedriver_win32_chrome83\chromedriver.exe")
    driver.get(url)
    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 2
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1
    needed_class_tag = "article-preview-mixed article-preview-mixed--secondary article-preview-mixed--with-absolute-secondary-item js-article-link"

    while i<scroll_time:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 

    ##### Extract Nur.kz URLs #####
    urls = []
    soup = BeautifulSoup(driver.page_source, "html.parser")
    for a_tag in soup.find_all("a", {"class":  needed_class_tag}):
        link = a_tag.attrs['href']
        urls.append(link)
  
    return urls    

In [33]:
f_links = scraper("https://www.nur.kz/nurfin/all/", 100)
print("Finance news scraped. Number of news: " + str(len(finance)))
p_links = scraper("https://www.nur.kz/politics/", 100)
print("Politics news scraped. Number of news: " + str(len(politics)))
w_links = scraper("https://www.nur.kz/world/", 100)
print("World news scraped. Number of news: " + str(len(world)))
s_links = scraper("https://www.nur.kz/society/",100)
print("Society news scraped. Number of news: " + str(len(society)))


C:\Users\anarb\AppData\Local\Temp\ipykernel_15100\187362017.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"E:\Chromedriver\chromedriver_win32_chrome83\chromedriver.exe")


Politics news scraped. Number of news: 840
World news scraped. Number of news: 900
Society news scraped. Number of news: 900


In [9]:
def get_news(links, category):
    column_names = ["text","category"]
    news = pd.DataFrame(columns = column_names)
    for link in links:
        response = requests.get(link)
        sleep(randint(1, 3))
        soup = BeautifulSoup(response.text, 'lxml')
        text = ""
        for item in soup.find_all("p",{"class": "align-left formatted-body__paragraph" }):
            text = text + item.text
        news.loc[len(news.index)] = [text, category]      
    return news


In [64]:
politics = get_news(politics, "Politics")
finance = get_news(finance, "Finance")
world = get_news(world, "World")
society = get_news(society, "Society")

In [2]:
politics = pd.read_csv('politics.csv')
finance = pd.read_csv('finance.csv')
society = pd.read_csv('society.csv')
world = pd.read_csv('world.csv')

In [11]:
df= pd.DataFrame({"text": politics['text'], "label":1 })
df = pd.concat([df, pd.DataFrame({"text": finance['text'], "label":2 })],ignore_index=True)
df = pd.concat([df, pd.DataFrame({"text": society['text'], "label":3 })],ignore_index=True)
df = pd.concat([df, pd.DataFrame({"text": world['text'], "label":4 })],ignore_index=True)

In [12]:
df

,text,label
0,Консульство Словацкой Республики открыли в Алм...,1
1,Отгрузка казахстанской нефти началась из турец...,1
2,Главы государств и международных организаций н...,1
3,Вопросы привлечения отечественных производител...,1
4,Согласно итогам дневных торгов на Казахстанско...,1
...,...,...
3475,Сильное землетрясение в окрестностях города Га...,4
3476,Руководитель отдела земельных отношений Майско...,4
3477,Посольство Казахстана в Турции на своей официа...,4
3478,На брифинге в правительстве глава Минсельхоза ...,4


In [13]:
import itertools
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
import string
from nltk.corpus import stopwords
import re

def text_clean(text):        
    text = text.lower() 
    text = re.sub('https?://\S+|www\.\S+', '', text) 
    text = re.sub(r"\b\d+\b", "", text) 
    text = re.sub('<.*?>+', '', text) 
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) 
    text = re.sub('\n', '', text)
    text = re.sub('[’“”…]', '', text)
    text = re.sub('–', '', text)   
    return text

def delete_stopwords(text):
    stop_words = set(stopwords.words('russian'))
    return ' '.join([word for word in text.split() if word not in (stop_words)])



In [14]:
df['text'] = df['text'].apply(lambda text: text_clean(str(text)))
df['text'] = df['text'].apply(lambda text: delete_stopwords(str(text)))

In [15]:
df

,text,label
0,консульство словацкой республики открыли алмат...,1
1,отгрузка казахстанской нефти началась турецког...,1
2,главы государств международных организаций нап...,1
3,вопросы привлечения отечественных производител...,1
4,согласно итогам дневных торгов казахстанской ф...,1
...,...,...
3475,сильное землетрясение окрестностях города гази...,4
3476,руководитель отдела земельных отношений майско...,4
3477,посольство казахстана турции своей официальной...,4
3478,брифинге правительстве глава минсельхоза объяс...,4


In [26]:
stop_words = set(stopwords.words('russian'))
stop_words

{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [27]:
from collections import Counter

vocab = Counter()
for text in df.text:
    for word in text.split(' '):
        vocab[word] += 1

vocab.most_common(20)

[('года', 4415),
 ('также', 3869),
 ('это', 3734),
 ('году', 2546),
 ('тенге', 2438),
 ('сегодня', 2159),
 ('казахстана', 1739),
 ('которые', 1672),
 ('время', 1665),
 ('алматы', 1649),
 ('страны', 1586),
 ('области', 1576),
 ('рк', 1504),
 ('казахстане', 1483),
 ('сша', 1407),
 ('могут', 1288),
 ('заявил', 1272),
 ('передает', 1271),
 ('сайте', 1240),
 ('курс', 1237)]

In [28]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
morph.parse('Новости')[0].normal_form

'новость'

In [29]:
def stemming(text):
    morph = pymorphy2.MorphAnalyzer()
    return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])

In [30]:
df.head()

,text,label
0,консульство словацкой республики открыли алмат...,1
1,отгрузка казахстанской нефти началась турецког...,1
2,главы государств международных организаций нап...,1
3,вопросы привлечения отечественных производител...,1
4,согласно итогам дневных торгов казахстанской ф...,1


In [34]:
from sklearn.base import BaseEstimator, TransformerMixin
class CustomTextPrep(BaseEstimator, TransformerMixin):
    """My custom description
    Args:
        delete_stopwords (bool): True if I want to delete most popular russian words 
    Returns:

    """
    def __init__(self, delete_stopwords=True):
        self.delete_stopwords = delete_stopwords
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X: pd.DataFrame, y=None) -> pd.DataFrame:
        def text_clean(text):        
            text = text.lower() 
            text = re.sub('https?://\S+|www\.\S+', '', text) 
            text = re.sub(r"\b\d+\b", "", text) 
            text = re.sub('<.*?>+', '', text) 
            text = re.sub('[%s]' % re.escape(string.punctuation), '', text) 
            text = re.sub('\n', '', text)
            text = re.sub('[’“”…]', '', text)

            return text

        def delete_stopwords(text):
            stop_words = set(stopwords.words('russian'))
            return ' '.join([word for word in text.split() if word not in (stop_words) and word not in ('также', "это", "которые")])
        def stemming(text):
            morph = pymorphy2.MorphAnalyzer()
            return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])

        X_copy = X.copy()
        X_copy = X_copy.apply(lambda text: text_clean(str(text)))
        if delete_stopwords:
            X_copy= X_copy.apply(lambda text: delete_stopwords(str(text)))
        
        X_copy = X_copy.apply(lambda text: stemming(str(text)))
        
        return X_copy

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'],test_size=0.20,random_state=42,shuffle=True)

sgd_pipeline = Pipeline(steps=[
    ('transformer',CustomTextPrep()),
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('sgd', SGDClassifier()),

])


sgd_pipeline.fit(x_train, y_train)

Pipeline(steps=[('transformer', CustomTextPrep()), ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()), ('sgd', SGDClassifier())])

In [36]:
from sklearn import metrics

score = metrics.accuracy_score(sgd_pipeline.predict(x_test), y_test)
score

0.9137931034482759

In [29]:
sgd_pipeline_1 = Pipeline(steps=[
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('sgd', SGDClassifier()),

])


sgd_pipeline_1.fit(x_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('sgd', SGDClassifier())])

In [30]:
score_1 = metrics.accuracy_score(sgd_pipeline_1.predict(x_test), y_test)
score_1

0.9252873563218391

In [32]:
import joblib

filename = "nlp-model-ru.joblib"

# save model
joblib.dump(sgd_pipeline_1, filename)

# load model
loaded_model = joblib.load(filename)

In [33]:
example_text = 'Просрочки по кредитам являются проблемой не только для клиентов банка, которым грозит блокировка счетов и штрафные санкции, но и для самих финансовых организаций. Наличие «плохих» кредитов негативно сказывается на качестве портфеля банка. Поэтому каждый банк стремится снизить объем «плохих» кредитов в своем портфеле. Так, по состоянию на 1 апреля 2023 года, сумма кредитов с просрочкой в банках второго уровня Казахстана составила 1,1 трлн тенге. Это на 7,4% больше, чем месяцем ранее. При этом количество «плохих» займов, оплату по которым просрочили на 90 дней и больше (NPL 90+), тоже увеличилось. Их количество в стране выросло на 2,5%, общая сумма составила 847,4 млрд тенге. Ссудный портфель банков второго уровня Казахстана при этом увеличился на 1,4% за месяц (до 24,6 трлн тенге), отмечается в статье. Таблица: finprom.kz Что касается доли займов с просрочкой свыше 30 дней, их доля среди общего объема кредитов в банках второго уровня Казахстана составила 4,6% (против 4,3% месяцем ранее). Доля NPL 90+, в свою очередь, составила 3,4% (показатель увеличился на 0,03 процентных пункта за месяц). При этом нарастить свой ссудный портфель за последний месяц смогли лишь шесть банков из топовой десятки банков второго уровня страны. Всего в данном секторе задействованы 13 БВУ. Из всей десятки крупнейших банков-кредиторов страны, уменьшить общий объем кредитов с просрочкой более 30 дней удалось лишь одной финансовой организации. Это Jusan Bank. Таковых кредитов в банке стало меньше на 2,8%. Общая сумма сократилась до 142 млрд тенге по итогам марта 2023 года, поделились аналитики. Читайте больше: https://www.nur.kz/nurfin/banks/2021305-kakim-bankam-kazahstana-udalos-sokratit-obem-plohih-kreditov-s-dolgoy-prosrochkoy/'

In [34]:
sgd_pipeline_1.predict(pd.Series([example_text]))
#finance case

array([2], dtype=int64)